In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# -------------------
# 1. 혼잡도 데이터
# -------------------
sub_df = pd.read_csv("../../data/TRA-SUBCONGEST/TRA-SUBCONGEST.csv")
time_cols = [col for col in sub_df.columns if "시" in col]
sub_df["station_mean_congest"] = sub_df[time_cols].mean(axis=1)

# -------------------
# 2. 역-구 매핑 데이터
# -------------------
map_df = pd.read_csv("../../data/TRA-SUBCONGEST/서울시 지하철역 엘리베이터 위치정보.csv", encoding='cp949')
map_df = map_df[["시군구명", "지하철역명"]].drop_duplicates()

# -------------------
# 3. 병합 (출발역 ↔ 지하철역명)
# -------------------
merged = pd.merge(sub_df, map_df, left_on="출발역", right_on="지하철역명", how="left")

# -------------------
# 4. 구별 평균 혼잡도
# -------------------
gu_stats = merged.groupby("시군구명")["station_mean_congest"].mean().reset_index()

# -------------------
# 5. MinMax Scaling (0~100점)
# -------------------
scaler = MinMaxScaler(feature_range=(0, 100))
gu_stats["subway_congest_score"] = scaler.fit_transform(gu_stats[["station_mean_congest"]])

# -------------------
# 6. 저장
# -------------------
gu_stats.to_csv("../../result/TRA/TRA-SUBCONGEST_result.csv", index=False, encoding="utf-8-sig")

print(gu_stats.head())

  시군구명  station_mean_congest  subway_congest_score
0  강남구             27.953609             56.388565
1  강동구             31.980339             71.769017
2  강서구             13.190612              0.000000
3  관악구             35.641699             85.753901
4  광진구             39.371446            100.000000
